In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch

In [ ]:
from utils import ActNorm, Invertibe1by1Convolution
import utils

In [ ]:
torch.manual_seed(42)
tmp_input = torch.randn(5, 3, 4, 4)
tmp_input[0, ...]

tensor([[[ 1.9269,  1.4873,  0.9007, -2.1055],
         [ 0.6784, -1.2345, -0.0431, -1.6047],
         [-0.7521,  1.6487, -0.3925, -1.4036],
         [-0.7279, -0.5594, -0.7688,  0.7624]],

        [[ 1.6423, -0.1596, -0.4974,  0.4396],
         [-0.7581,  1.0783,  0.8008,  1.6806],
         [ 1.2791,  1.2964,  0.6105,  1.3347],
         [-0.2316,  0.0418, -0.2516,  0.8599]],

        [[-1.3847, -0.8712, -0.2234,  1.7174],
         [ 0.3189, -0.4245,  0.3057, -0.7746],
         [-1.5576,  0.9956, -0.8798, -0.6011],
         [-1.2742,  2.1228, -1.2347, -0.4879]]])

In [ ]:
ac = ActNorm()
forwarded, logdet = ac(tmp_input)

In [ ]:
forwarded[0, ...]

tensor([[[ 1.7572,  1.3327,  0.7663, -2.1364],
         [ 0.5517, -1.2954, -0.1450, -1.6528],
         [-0.8296,  1.4886, -0.4823, -1.4586],
         [-0.8062, -0.6435, -0.8457,  0.6328]],

        [[ 1.5180, -0.3344, -0.6816,  0.2816],
         [-0.9497,  0.9382,  0.6529,  1.5574],
         [ 1.1446,  1.1624,  0.4572,  1.2018],
         [-0.4084, -0.1274, -0.4289,  0.7136]],

        [[-1.2794, -0.7635, -0.1126,  1.8373],
         [ 0.4322, -0.3147,  0.4190, -0.6664],
         [-1.4531,  1.1122, -0.7721, -0.4922],
         [-1.1684,  2.2446, -1.1287, -0.3784]]], grad_fn=<SelectBackward0>)

In [ ]:
logdet

tensor([-0.0433, -0.0433, -0.0433, -0.0433, -0.0433],
       grad_fn=<RepeatBackward0>)

In [ ]:
ac.batch_variance

Parameter containing:
tensor([[[[1.0592]],

         [[0.9344]],

         [[0.9782]]]], requires_grad=True)

In [ ]:
ac.batch_mean

Parameter containing:
tensor([[[[ 0.1071]],

         [[ 0.1657]],

         [[-0.1113]]]], requires_grad=True)

In [ ]:
torch.log(ac.batch_variance)

tensor([[[[ 0.0575]],

         [[-0.0678]],

         [[-0.0220]]]], grad_fn=<LogBackward0>)

In [ ]:
torch.log(ac.batch_std**2)

tensor([[[[ 0.0701]],

         [[-0.0552]],

         [[-0.0094]]]], grad_fn=<LogBackward0>)

In [ ]:
ac.batch_variance.sqrt().log(), (ac.batch_std).log()

(tensor([[[[ 0.0288]],
 
          [[-0.0339]],
 
          [[-0.0110]]]], grad_fn=<LogBackward0>),
 tensor([[[[ 0.0350]],
 
          [[-0.0276]],
 
          [[-0.0047]]]], grad_fn=<LogBackward0>))

In [ ]:
torch.manual_seed(41)
inv_conv = Invertibe1by1Convolution(3)

inv_conv.weight

Parameter containing:
tensor([[[[-0.8341]],

         [[-0.0222]],

         [[-0.5512]]],


        [[[-0.1638]],

         [[-0.9441]],

         [[ 0.2860]]],


        [[[-0.5268]],

         [[ 0.3288]],

         [[ 0.7838]]]], requires_grad=True)

In [ ]:
forwarded, log_det = inv_conv(tmp_input)
log_det

tensor(1.9073e-06, grad_fn=<MulBackward0>)

In [ ]:
_, _, h, w = tmp_input.shape

In [ ]:
h * w * torch.log(torch.abs(torch.linalg.det(inv_conv.weight.squeeze())))

tensor(1.9073e-06, grad_fn=<MulBackward0>)

In [ ]:
h * w * torch.slogdet(inv_conv.weight.squeeze())[1]

tensor(9.5367e-07, grad_fn=<MulBackward0>)

In [ ]:
torch.manual_seed(41)
tmp_input = torch.randn(5, 4, 4, 4)
gAff = utils.GlowAffineCouplingLayer(input_shape=(4, 4, 4), filters=3)
y, log_det, log_scale = gAff(tmp_input)

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
reversed = gAff.reverse(y)
torch.linalg.norm(reversed - tmp_input)

tensor(2.4767e-07, grad_fn=<LinalgVectorNormBackward0>)

In [ ]:
torch.manual_seed(41)
tmp_input = torch.randn(5, 4, 4, 4)
flow = utils.StepOfGlow(base_input_shape=tmp_input.shape[1:], num_filters=512)

In [ ]:
forwarded, log_det = flow(tmp_input)

In [ ]:
log_det

tensor([-2.6153, -2.6153, -2.6153, -2.6153, -2.6153], grad_fn=<AddBackward0>)

In [ ]:
backwarded = flow.reverse(forwarded)

tensor([[[-1.0897e-01, -5.3170e-01,  1.3801e+00, -2.5943e+00],
         [-1.6551e+00,  1.1393e+00, -1.3062e+00, -5.1382e-01],
         [ 1.0265e+00,  1.9025e-01, -1.2840e-01, -1.9024e-01],
         [ 9.7518e-01, -3.3117e-01,  2.1830e+00, -9.8944e-01]],

        [[ 3.4584e-01,  1.1045e+00,  2.0812e-01,  1.1383e+00],
         [-1.0883e-01,  1.7641e+00, -2.0232e-01, -1.2534e+00],
         [ 1.4874e+00, -3.6911e-01,  4.9998e-02,  9.0914e-01],
         [-3.3591e-01, -7.1173e-01, -2.4907e-01, -1.0833e+00]],

        [[ 1.1410e+00,  1.3601e+00, -7.7837e-01, -1.4611e-01],
         [ 2.0146e-02, -2.2777e-01, -4.1102e-02,  8.5986e-01],
         [ 1.0888e+00,  4.0737e-01, -7.1665e-01, -1.8994e-01],
         [-1.0085e+00, -1.1568e+00,  8.9326e-01, -1.1979e+00]],

        [[-3.3335e-01, -1.5872e+00, -1.2233e+00, -1.5227e-03],
         [ 6.1887e-01, -7.3106e-01,  2.4905e-01,  1.1513e+00],
         [-6.1837e-01, -1.7308e+00, -9.2443e-01,  1.2141e-01],
         [ 4.5520e-01, -1.0203e-01,  7.4635e-01, 

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
torch.linalg.norm(backwarded - tmp_input)

tensor(1.5357e-06, grad_fn=<LinalgVectorNormBackward0>)

In [ ]:
1